### Gridworld value evaluation

We consider a 5x5 grid as the environment and each position in the grid (row, column) defines a different state. An agent is allowed to move one block in any if the four directions (north, south, east, west). A (0,1) and B(0,3) are special states, ie, if the agent reaches A, it will get a reward of +10 and transport directly to A'(4,1) and if it reaches B, it will get a reward of +5 and transport directly to B'(2,3).
If the agent tries to move off the edge, it will get a reward of -1, but will continue to stay within the grid.
The agent is allowed to choose any direction with uniform probability. i.e, a uniform random policy is used.

The goal is to compute the state-valye function for all states under the given policy, which tells us the expected return (future cumulative discounted reward) starting from each state.
We use iterative policy evaluation, applying the Bellman expectation equation repeatedly until the values converge.

In [1]:
import numpy as np

In [8]:
grid_size = 5
states = [(i,i) for i in range(grid_size) for j in range(grid_size)]
actions = [(-1,0),(1,0),(0,-1),(0,1)]
actions_prob = 0.25
gamma = 0.9
theta = 0.0001

specials = {
    (0,1): ((4,1),10),
    (0,3):((2,3),5)
}

v_pi = np.zeros((grid_size, grid_size))


In [5]:
next_state, reward = specials[(0,1)]
print(f"Special state: {next_state}, reward: {reward}") 

Special state: (4, 1), reward: 10


In [6]:
def step(state,action):
    """
    Takes state and action as input
    returns the next state and reward
    reward = +10 if state is (0,10) and +5 if state is (0,3)
    reward = -1 if action would lead to out of bounds
    reward = 0 otherwise
    """
    if state in specials:
        state_next, reward = specials[state]
        return state_next, reward
    i, j = state
    di, dj = action
    ni, nj = i + di, j + dj

    if 0<= ni < grid_size and 0 <= nj < grid_size:
        return (ni,nj), 0
    else:
        return state,-1
    
    

In [11]:
iteration = 0
while True:
    delta = 0
    new_v = np.zeros_like(v_pi)

    for i in range(grid_size):
        for j in range(grid_size):
            s = (i, j)
            value = 0
            for a in actions:
                next_s, r = step(s, a)
                ni, nj = next_s
                value += actions_prob * (r + gamma * v_pi[ni, nj])
            new_v[i, j] = value
            delta = max(delta, abs(v_pi[i, j] - value))

    v_pi[:] = new_v
    iteration += 1
    if delta < theta:
        break

# Print final value function
print("Value function after", iteration, "iterations:")
print(np.round(v_pi, 2))


Value function after 1 iterations:
[[ 3.31  8.79  4.43  5.32  1.49]
 [ 1.52  2.99  2.25  1.91  0.55]
 [ 0.05  0.74  0.67  0.36 -0.4 ]
 [-0.97 -0.43 -0.35 -0.58 -1.18]
 [-1.86 -1.34 -1.23 -1.42 -1.97]]
